In [1]:
import math
import os
import sys
import pandas as pd
sys.path.append("/home/weber/PycharmProjects/EXOTIC/src")
from statannot import add_stat_annotation

pd.options.mode.chained_assignment = None  # default='warn'
import multiprocessing
import parmap
import numpy as np
import collections
from tqdm import tqdm
import pandarallel
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=20, progress_bar=True)
# tqdm.pandas()
from pprint import pprint
from scipy.stats import zscore
from scipy import stats

import requests
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mc
from matplotlib.colors import LogNorm
import json
import matplotlib.ticker as mtick
from utils.utils import load_config_file

## YAML FILES CONFIG
yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/src/config.yaml")
exotic_files = yaml

import matplotlib.font_manager as font_manager

font_dirs = ['/home/weber/Fonts', ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['font.weight'] = 'light'

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:42: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.


In [4]:
drugbank_kirsley = pd.read_excel('/gstock/biolo_datasets/DrugBank/drugbank_kirsley.xlsx')
drugbank_kirsley

,drug_id,drug_name,effect_type,gene,rsid,uniprot_acc,pmid,change,desc,ensg,varid
0,DB00002,Cetuximab,snp_effect,FCGR2A,rs1801274,P12318,17704420.0,H allelle,Patients with this genotype may have increased...,ENST00000367972,chr1:g.161479745A>T
1,DB00002,Cetuximab,snp_effect,FCGR3A,rs396991,P08637,17704420.0,G > T,Patients with this genotype have increased pro...,ENST00000443193,chr1:g.161514542A>T
2,DB00008,Peginterferon alfa-2a,snp_effect,IFNL3,rs12979860,Q8IZI9,24096968.0,C > T,Patients with this genotype in IFNL3 have a re...,ENST00000413851,chr19:g.39738787C>T
3,DB00022,Peginterferon alfa-2b,snp_effect,IFNL3,rs12979860,Q8IZI9,24096968.0,C > T,Patients with this genotype in IFNL3 have a re...,ENST00000413851,chr19:g.39738787C>T
4,DB00065,Infliximab,snp_effect,FCGR3A,rs1801274,P08637,18930989.0,CC Allele (homozygous),Patients with this genotype have increased ACR...,ENST00000443193,chr1:g.161479745A>T
...,...,...,...,...,...,...,...,...,...,...,...
319,DB09256,Tegafur,adverse_reaction,DPYD,rs67376798,Q12882,23988873.0,T > A,The presence of this genotype in DPYD may be a...,ENST00000370192,chr1:g.97547947T>A
320,DB09256,Tegafur,adverse_reaction,DPYD,rs1801158,Q12882,23988873.0,G > A,The presence of this genotype in DPYD may be a...,ENST00000370192,chr1:g.97981421C>T
321,DB09256,Tegafur,adverse_reaction,DPYD,rs1801159,Q12882,23988873.0,A > G,The presence of this genotype in DPYD may be a...,ENST00000370192,chr1:g.97981395T>C
322,DB09256,Tegafur,adverse_reaction,DPYD,rs1801160,Q12882,23988873.0,G > A,The presence of this genotype in DPYD may be a...,ENST00000370192,chr1:g.97770920C>T


In [5]:
exotic = pd.read_parquet('/gstock/EXOTIC/data/EXOTIC/EXOTIC_modified_zscore_mean_prop_corrected.parquet')
exotic

,symbol,ensg,HGNC,Exon,Ratio_num,mRNA_nb,mRNA_nb_total,MAP,mean_proportion,Adipose - Subcutaneous,...,Vagina,Whole Blood,EXOTIC_down,EXOTIC_bins_down,EXOTIC_bins_lite_down,EXOTIC_tissues_down,EXOTIC_up,EXOTIC_bins_up,EXOTIC_bins_lite_up,EXOTIC_tissues_up
0,A2ML1,ENSG00000166535,23336,8975248-8975309,0.500000,1,2,A2ML1_8975248-8975309,0.69514,0.578349,...,0.500000,0.578349,0.713205,0.7 - 0.75,0.7 - 0.8,[Brain - Cerebellar Hemisphere],0.578349,0.55 - 0.6,0.5 - 0.6,"[Adipose - Subcutaneous, Adipose - Visceral (O..."
1,A2ML1,ENSG00000166535,23336,8975778-8975961,0.500000,1,2,A2ML1_8975778-8975961,0.69514,0.578349,...,0.500000,0.578349,0.713205,0.7 - 0.75,0.7 - 0.8,[Brain - Cerebellar Hemisphere],0.578349,0.55 - 0.6,0.5 - 0.6,"[Adipose - Subcutaneous, Adipose - Visceral (O..."
2,A2ML1,ENSG00000166535,23336,8976316-8976478,0.500000,1,2,A2ML1_8976316-8976478,0.69514,0.578349,...,0.500000,0.578349,0.713205,0.7 - 0.75,0.7 - 0.8,[Brain - Cerebellar Hemisphere],0.578349,0.55 - 0.6,0.5 - 0.6,"[Adipose - Subcutaneous, Adipose - Visceral (O..."
3,A2ML1,ENSG00000166535,23336,8982323-8982375,0.500000,1,2,A2ML1_8982323-8982375,0.69514,0.578349,...,0.500000,0.578349,0.713205,0.7 - 0.75,0.7 - 0.8,[Brain - Cerebellar Hemisphere],0.578349,0.55 - 0.6,0.5 - 0.6,"[Adipose - Subcutaneous, Adipose - Visceral (O..."
4,A2ML1,ENSG00000166535,23336,8987258-8987278,0.500000,1,2,A2ML1_8987258-8987278,0.69514,0.578349,...,0.500000,0.578349,0.713205,0.7 - 0.75,0.7 - 0.8,[Brain - Cerebellar Hemisphere],0.578349,0.55 - 0.6,0.5 - 0.6,"[Adipose - Subcutaneous, Adipose - Visceral (O..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27547,ZXDC,ENSG00000070476,28160,126158475-126158570,0.500000,1,2,ZXDC_126158475-126158570,0.20989,0.500250,...,0.508499,0.495250,0.542398,0.5 - 0.55,0.5 - 0.6,[Brain - Nucleus accumbens (basal ganglia)],0.574443,0.55 - 0.6,0.5 - 0.6,[Muscle - Skeletal]
27548,ZXDC,ENSG00000070476,28160,126160608-126160789,0.500000,1,2,ZXDC_126160608-126160789,0.20989,0.500250,...,0.508499,0.495250,0.542398,0.5 - 0.55,0.5 - 0.6,[Brain - Nucleus accumbens (basal ganglia)],0.574443,0.55 - 0.6,0.5 - 0.6,[Muscle - Skeletal]
27549,ZXDC,ENSG00000070476,28160,126178496-126178580,0.500000,1,2,ZXDC_126178496-126178580,0.20989,0.500250,...,0.508499,0.495250,0.542398,0.5 - 0.55,0.5 - 0.6,[Brain - Nucleus accumbens (basal ganglia)],0.574443,0.55 - 0.6,0.5 - 0.6,[Muscle - Skeletal]
27550,ZXDC,ENSG00000070476,28160,126180378-126181063,0.500000,1,2,ZXDC_126180378-126181063,0.34803,0.515245,...,0.505750,0.441272,0.558728,0.55 - 0.6,0.5 - 0.6,[Whole Blood],0.606828,0.6 - 0.65,0.6 - 0.7,[Artery - Tibial]


In [41]:

l = list()
import re
for gene in exotic.symbol.unique().tolist():
    exotic_gene = exotic.loc[exotic['symbol'] == gene]
    drug_gene = drugbank_kirsley.loc[drugbank_kirsley['gene'] == gene]
    drug_gene = drug_gene.loc[drug_gene['varid'].isna() == False]
    
    if drug_gene.empty is False:
        print(gene)
        drug_gene['var_pos'] = drug_gene['varid'].apply(lambda r: re.search('\d{3,}', r).group(0))
        drug_gene['var_pos'] = drug_gene['var_pos'].astype(int)
        for exon in exotic_gene.Exon.values:
            between = drug_gene.var_pos.between(int(exon.split('-')[0]), int(exon.split('-')[1]))
            if True in between.values.tolist():
                drug_gene_filter = drug_gene.loc[list(between.index)]
                drug_gene_filter['Exon'] = exon
                l.append(drug_gene_filter)
drug_df_filter = pd.concat(l)
drug_df_filter
            

ABCB1
ABCG2
ACE
COMT
CREB1
DPYD
EGFR
ERBB2
FCGR2A
FCGR3A
FLOT1
GRIK2
GRIK4
HMGCR
HNRNPD
KCNH2
MTHFR
RARG
RRM1
SERPINC1
TYMS
VKORC1


,drug_id,drug_name,effect_type,gene,rsid,uniprot_acc,pmid,change,desc,ensg,varid,var_pos,Exon
123,DB00544,Fluorouracil,adverse_reaction,DPYD,rs1801265,Q12882,17848752.0,C allele,Patients with this genotype have reduced metab...,ENST00000370192,chr1:g.98348885T>C,98348885,97547886-97548026
125,DB00544,Fluorouracil,adverse_reaction,DPYD,rs3918290,Q12882,18299612.0,A allele,Patients with this genotype have reduced metab...,ENST00000370192,chr1:g.97915614C>G,97915614,97547886-97548026
126,DB00544,Fluorouracil,adverse_reaction,DPYD,rs67376798,Q12882,23603345.0,T > A,Patients with this genotype have reduced metab...,ENST00000370192,chr1:g.97547947T>A,97547947,97547886-97548026
127,DB00544,Fluorouracil,adverse_reaction,DPYD,rs3918290,Q12882,23988873.0,G > A,The presence of this genotype in DPYD is assoc...,ENST00000370192,chr1:g.97915614C>G,97915614,97547886-97548026
128,DB00544,Fluorouracil,adverse_reaction,DPYD,rs55886062,Q12882,23988873.0,A > C,The presence of this genotype in DPYD is assoc...,ENST00000370192,chr1:g.97981343A>T,97981343,97547886-97548026
...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,DB08916,Afatinib,snp_effect,EGFR,rs28929495,P00533,15118073.0,G > A or C or T,The presence of this polymorphism in EGFR is a...,ENST00000420316,chr7:g.55241707G>A,55241707,55259412-55259567
5,DB00072,Trastuzumab,adverse_reaction,ERBB2,rs1136201,P04626,17693647.0,"G Allele, heterozygote",Patients with this genotype have increased ris...,ENST00000584601,chr17:g.37879588A>C,37879588,37879572-37879710
274,DB01233,Metoclopramide,snp_effect,KCNH2,rs1805123,Q12809,22688145.0,A > C,The presence of this polymorphism in KCNH2 may...,ENST00000330883,chr7:g.150645534T>C,150645534,150645532-150645631
280,DB01233,Metoclopramide,adverse_reaction,KCNH2,rs3815459,Q12809,22688145.0,G > A,The presence of this polymorphism in KCNH2 is ...,ENST00000330883,chr7:g.150644394C>A,150644394,150645532-150645631


In [55]:
map_dict = {
    "se_blood" : ["Whole Blood"],
    "se_heart" : ["Heart - Atrial Appendage", "Heart - Left Ventricle"],
    "se_congenital" : [""],
    "se_ear" : [""],
    "se_endocrine" : ["Thyroid", "Pituitary", "Adrenal Gland"],
    "se_eye" : [""],
    "se_gi" : [""],
    "se_hepatobiliary" : [""],
    "se_immune" : ["Cells - EBV-transformed lymphocytes"],
    "se_infection" : ["Cells - EBV-transformed lymphocytes"],
    "se_metabolism" : [""],
    "se_musculoskeletal" : ["Muscle - Skeletal"],
    "se_neoplasm" : [""],
    "se_nervous" : ["Nerve - Tibial"],
    "se_pregnancy" : [""],
    "se_mental" : ["Brain - Amygdala", "Brain - Anterior cingulate cortex (BA24)", "Brain - Caudate (basal ganglia)", "Brain - Cerebellar Hemisphere", "Brain - Cerebellum", "Brain - Cortex", "Brain - Frontal Cortex (BA9)", "Brain - Hippocampus", "Brain - Hypothalamus", "Brain - Nucleus accumbens (basal ganglia)", "Brain - Putamen (basal ganglia)", "Brain - Spinal cord (cervical c-1)", "Brain - Substantia nigra"],
    "se_urologic" : ["Prostate", "Uterus", "Vagina", "Bladder"],
    "se_repro" : ["Cervix - Ectocervix", "Cervix - Endocervix", "Fallopian Tube", "Ovary", "Testis"],
    "se_respiratory" : ["Lung"],
    "se_skin" : ["Skin - Sun Exposed (Lower leg)", "Skin - Not Sun Exposed (Suprapubic)"],
    "se_vascular" : ["Artery - Aorta", "Artery - Coronary"],

}

In [73]:
drugbank_se = pd.read_excel('/gstock/biolo_datasets/DrugBank/publi_SE_drugs.xlsx')
drugbank_se = drugbank_se.loc[drugbank_se['has_se'] == True]
drugbank_se = drugbank_se.filter(regex='myDrugName|se_').melt(id_vars='myDrugName', value_vars=['se_blood', 'se_heart', 'se_congenital', 'se_ear',
       'se_endocrine', 'se_eye', 'se_gi', 'se_hepatobiliary', 'se_immune',
       'se_infection', 'se_metabolism', 'se_musculoskeletal', 'se_neoplasm',
       'se_nervous', 'se_pregnancy', 'se_mental', 'se_urologic', 'se_repro',
       'se_respiratory', 'se_skin', 'se_vascular'])
drugbank_se = drugbank_se.loc[drugbank_se['value'] == 1]
drugbank_se['Tissues'] = drugbank_se['variable'].map(map_dict)
drugbank_se = drugbank_se.explode('Tissues')
drugbank_se = drugbank_se.rename({'myDrugName' : 'drug_name'}, axis=1)
drugbank_se['drug_name'] = drugbank_se['drug_name'].astype(str)
# drugbank_se = pd.merge(drugbank_se,  drugbank_kirsley[['drug_name', 'gene']].drop_duplicates().groupby('drug_name')['gene'].apply(list).reset_index(), on='drug_name')
print(len(sorted(drugbank_se.drug_name.unique().tolist())))

1509


In [66]:
drugbank_kirsley_to_merge = drugbank_kirsley[['drug_name', 'gene']].drop_duplicates().groupby('drug_name')['gene'].apply(list).reset_index()
drugbank_kirsley_to_merge['drug_name'] = drugbank_kirsley_to_merge['drug_name'].str.lower()
drugbank_kirsley_to_merge


,drug_name,gene
0,abacavir,[HCP5]
1,acenocoumarol,"[CYP2C9, VKORC1]"
2,acetylsalicylic acid,"[CYP2C9, ITGB3, LTC4S]"
3,afatinib,[EGFR]
4,allopurinol,[HCP5]
...,...,...
107,venlafaxine,"[CYP2D6, ABCB1]"
108,verapamil,[ADRB1]
109,voriconazole,[CYP2C19]
110,warfarin,"[CYP2C9, VKORC1, CP4F2]"


In [76]:
pd.options.display.max_rows = 100
pd.merge(drug_df_filter, exotic, on='Exon').loc[90:92].to_dict()

{'drug_id': {90: 'DB00072', 91: 'DB01233', 92: 'DB01233'},
 'drug_name': {90: 'Trastuzumab', 91: 'Metoclopramide', 92: 'Metoclopramide'},
 'effect_type': {90: 'adverse_reaction',
  91: 'snp_effect',
  92: 'adverse_reaction'},
 'gene': {90: 'ERBB2', 91: 'KCNH2', 92: 'KCNH2'},
 'rsid': {90: 'rs1136201', 91: 'rs1805123', 92: 'rs3815459'},
 'uniprot_acc': {90: 'P04626', 91: 'Q12809', 92: 'Q12809'},
 'pmid': {90: 17693647.0, 91: 22688145.0, 92: 22688145.0},
 'change': {90: 'G Allele, heterozygote', 91: 'A > C', 92: 'G > A'},
 'desc': {90: 'Patients with this genotype have increased risk of cardiotoxicity with trastuzumab.',
  91: 'The presence of this polymorphism in KCNH2 may be associated with increased clinical efficacy with metoclopramide.',
  92: 'The presence of this polymorphism in KCNH2 is associated with higher incidences of adverse events from metoclopramide treatment.'},
 'ensg_x': {90: 'ENST00000584601',
  91: 'ENST00000330883',
  92: 'ENST00000330883'},
 'varid': {90: 'chr17:g.